In [1]:
import numpy as np
def troca_linhas(A, i, j):
    A[[i, j]] = A[[j, i]]

def multiplicar_linha(A, i, l):
    A[i] *= l

def soma_linha(A, i, j, a):
    A[i] += a * A[j]

def escalonar(A):
    n_linhas, n_colunas = A.shape
    linha_atual = 0

    for col in range(n_colunas):
        
        # Procurar o máximo na coluna atual (para pivotamento parcial)
        max_linha = linha_atual + np.argmax(np.abs(A[linha_atual:, col]))
        
        # Se o maior valor for zero, continue para a próxima coluna
        if A[max_linha, col] == 0:
            continue
        
        # Trocar a linha atual com a linha do pivô
        troca_linhas(A, linha_atual, max_linha)
        
        # Fazer com que o pivô seja 1
        multiplicar_linha(A, linha_atual, 1 / A[linha_atual, col])
        
        # Zerar os elementos abaixo do pivô
        for i in range(linha_atual + 1, n_linhas):
            soma_linha(A, i, linha_atual, -A[i, col])
            
        for i in range(linha_atual):
            soma_linha(A, i, linha_atual, -A[i, col])    

        linha_atual += 1  # Mover para a próxima linha
    
    A = A.T
    n_linhas, n_colunas = n_colunas, n_linhas
    
    # Contar o número de linhas não nulas (posto)
    posto = 0
    for i in range(n_linhas):
        if np.sum(A[i]) == 0:# Verifica se a linha não é nula
            continue
        posto += 1

    return posto == np.linalg.matrix_rank(A)

def LI_LD(*vetores):
    A = np.matrix(vetores).T # Matriz já sendo transposta
    if escalonar(A):
        print("É LI")
    else:
        print("É LD")
        
# Exemplo de uso

# Definir uma matriz
u1=[1., 2., 0, 0., -1., 0., 6., 0.2, 1.3, -5., 2.1, -2.]
u2=[2., 0.3, 0, 0., -2., 0., -3., -0.1, 1.5, -2., 3.7, -5.]
u3=[0., 0.7, 0, 0., -1., 0., 2., 0.6, 3.5, 0., 1.2, 3.]
u4=[5., 8., 0, 0., 2.3, 0., -2.5, 0.3, -0.5, 0., 0., 0.]
u5=[3., 3., 0, 0., 0.2, 0., -1.5, 1.3, -1.5, 0., 8., 0.]

v1=[2.7, 2.04, 0., 0., 0.35, 0., 0.35, 2.36, -4.29, 0., 14.21, 1.2]
v2=[5.9, 5.3, 0., 0., -7.62, 0., 3.6, 2.26, 13.81, -7.5, 15.67, -2.4]
v3=[33., 45.81, 0., 0., 2.2, 0., -6.9, 2.38, 9.15, -16., 16.86, -15.1]
v4=[3., -0.21, 0., 0., -4.7, 0., -8.6, 0.62, 7.65, 1., 7.34, -2.9]
v5=[-16.2, -24.67, 0., 0., -11.15, 0., 17.55, -4.69, 9.13, -13.7, -14.27, -13.7]

vetores = [
    (u1,u2,u3,u4,u5),
    (v1,v2,v3,v4,v5),
    ([1, 0, 0], [0, 1, 0], [0, 0, 1]),  # Caso 1
    ([1, 2, 3], [2, 4, 6], [3, 6, 9]),  # Caso 2
    ([1, 2, 3], [4, 5, 6], [7, 8, 9]),  # Caso 3
    ([1, 2, 3], [4, 5, 6], [0, 0, 0]),  # Caso 4
    ([1, 4, 7], [2, 5, 8], [3, 6, 10]), # Caso 5
    ([1, 2, 3, 4], [2, 4, 6, 8], [0, 1, 0, 1], [3, 6, 9, 12]), # Caso 6
    ([2, 4, 1], [1, 2, 0], [0, 0, 0])  # Caso 7
]

for i, v in enumerate(vetores):
    LI_LD(*v)


É LI
É LI
É LI
É LD
É LD
É LI
É LD
É LD
É LI


In [2]:
def verifica_span(u,v):
    U = np.column_stack((u)).astype(float)
    V = np.column_stack((v)).astype(float)
    if escalonar(U) and escalonar(V):
        print("O span de U é igual ao span de V")
    else:
        print("O span de U não é igual ao span de V")

verifica_span(vetores[0],vetores[1])
verifica_span(vetores[1],vetores[2])
verifica_span(vetores[2],vetores[3])
verifica_span(vetores[3],vetores[4])

O span de U é igual ao span de V
O span de U é igual ao span de V
O span de U não é igual ao span de V
O span de U não é igual ao span de V


In [101]:
def resolve_sistema(matriz, valores):
    n_linhas, n_colunas = matriz.shape
    
    # Aumenta a matriz com o vetor de valores
    matriz = np.hstack((matriz, valores))

    linha_atual = 0
    for col in range(n_colunas):
        if linha_atual < n_linhas:
            max_linha = linha_atual + np.argmax(np.abs(matriz[linha_atual:, col]))
        else:
            continue
        
        if matriz[max_linha, col] == 0:
            continue
        
        troca_linhas(matriz, linha_atual, max_linha)
        
        multiplicar_linha(matriz, linha_atual, 1 / matriz[linha_atual, col])
        
        for i in range(linha_atual + 1, n_linhas):
            soma_linha(matriz, i, linha_atual, -matriz[i, col])
        
        for col in range(n_colunas - 1, -1, -1):
            for i in range(col - 1, -1, -1):
                soma_linha(matriz, i, col, -matriz[i, col]) 

        linha_atual += 1
    
    # A solução estará na última coluna da matriz aumentada
    return matriz[:, -1]

def matriz_mudancaBase(B1, B2):
    B1 = np.array(B1).astype(float).T
    B2 = np.array(B2).astype(float).T
    n = B1.shape[1]
    
    P = np.zeros_like(B1, dtype=float)
    
    for i in range(n):
        # Pegue a i-ésima coluna de B1 e converta em uma matriz coluna 2D
        vetores = B1[:, [i]]
        
        # Copie B2 para não alterá-lo diretamente
        matriz = B2.copy()
        
        # Resolva o sistema linear
        solucao = resolve_sistema(matriz, vetores)
        
        # Coloque a solução na i-ésima coluna da matriz P
        P[:, i] = solucao.ravel()
           
    return P.T


print(matriz_mudancaBase(vetores[1], vetores[0]))
print(matriz_mudancaBase(vetores[0], vetores[1]))

[[ 2.00000000e-01 -5.00000000e-01 -3.00000000e-01 -5.00000000e-01
   2.00000000e+00  0.00000000e+00  0.00000000e+00 -2.22044605e-16
  -1.77635684e-15  5.55111512e-17  3.33066907e-16  0.00000000e+00]
 [ 7.00000000e-01  2.00000000e+00  3.00000000e+00  7.21644966e-16
   4.00000000e-01  0.00000000e+00  0.00000000e+00 -6.66133815e-16
   5.55111512e-16  1.38777878e-16 -9.99200722e-16  0.00000000e+00]
 [ 2.00000000e+00  3.00000000e+00  1.30000000e+00  5.00000000e+00
   3.45960605e-16  0.00000000e+00  0.00000000e+00 -1.22453014e-16
   3.95462294e-15 -2.73629859e-15 -1.63199973e-15  0.00000000e+00]
 [-1.00000000e+00  2.00000000e+00  1.70000000e+00  7.39491047e-16
   9.82598926e-17  0.00000000e+00  0.00000000e+00 -6.08967322e-17
   3.06009654e-15  2.82505682e-16 -1.06324836e-15  0.00000000e+00]
 [ 2.10000000e+00  1.60000000e+00 -5.00000000e-01 -2.50000000e+00
  -3.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.77635684e-15  5.55111512e-16 -4.66293670e-15  0.00000000e+00]]
[[ 1

In [102]:
def encontrar_coeficientes(B1, B2, resultado):
    # Passo 1: Calcular a matriz de transformação de base de B1 para B2
    P = matriz_mudancaBase(B1, B2)
    
    # Passo 2: Resolver o sistema U * [a1, a2, a3, a4, a5]^T = resultado
    a = resolve_sistema(B1.T, resultado.T)
    
    # Passo 3: Calcular os coeficientes b usando b = P * a
    b = P@a
    
    return a, b

linha = np.array([[3.7, -0.79, 0., 0., -4.12, 0., 1.3, 3.45, -2.84, -4.5, 27.08, -4.5]])

encontrar_coeficientes(np.array(vetores[1]),np.array(vetores[0]),linha)

(array([ 2.01781718e+00, -1.20938965e-01,  8.63248454e-02,  4.82636987e-01,
         3.29086256e-01,  0.00000000e+00,  0.00000000e+00,  1.33226763e-15,
        -1.33226763e-15, -4.44089210e-16,  0.00000000e+00, -2.91433544e-16]),
 array([ 0.85498948,  1.56120313,  6.19822469, -2.11294287,  1.80690007]))